## Data Collection and cleaning

In [2]:
# imports
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import datetime
from pathlib import Path
from MCForecastTools import MCSimulation
import pylab
from matplotlib import pyplot
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# load env file
load_dotenv("LT.env")

True

#### Using Alpaca to retrieve Gold prices

In [4]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = 'v2'
)

# initialize variables for api
ticker = "GLD"
timeframe = "1D"
current_date = datetime.datetime.now()
start_date = datetime.datetime(current_date.year-5, current_date.month, current_date.day)

# read in data
ticker_data = api.get_barset(
    ticker,
    timeframe,
    start=start_date,
    end=current_date,
    limit=1000
).df

# save Gold Data as a CSV
ticker_data.to_csv('Resources/gold_data.csv')

#### Manipulating BTC data

In [5]:
btc_filepath = Path("Resources/bitstampUSD_1-min_data_2012-01-01_to_2021-03-31.csv")
btc_data = pd.read_csv(btc_filepath)
btc_data.head()

# clean data
btc_data = btc_data.dropna()
btc_data.isnull().sum()

btc_data.Timestamp = pd.to_datetime(btc_data.Timestamp, unit='s')

# resampling to daily frequency
btc_data.index = btc_data.Timestamp
btc_data = btc_data.resample('D').mean()

# saving to a CSV file
btc_data.to_csv('cleaned_btc_data.csv')

#### Final manipulations and concatenation of files

In [6]:
#Read BTC Data; drop columns not used and adjust header to match data from Alpaca
btc_path = Path("Resources/cleaned_btc_data.csv")
btc = pd.read_csv(btc_path, index_col="Timestamp",parse_dates=True, infer_datetime_format=True).drop(['Open', 'High', 'Low', 'Volume_(BTC)', 'Volume_(Currency)', 'Weighted_Price'], 1)
btc.columns = pd.MultiIndex.from_tuples([('BTC', 'close')])
btc.head()

,BTC
,close
Timestamp,
2011-12-31,4.482500
2012-01-01,4.806667
2012-01-02,5.000000
2012-01-03,5.252500
2012-01-04,5.223333


In [13]:
#Read Gold data, drop columns not used
gold_path = Path("Resources/gold_data.csv")
gold = pd.read_csv(gold_path, parse_dates=True, infer_datetime_format=True, index_col=0, header=[0,1])
gold = gold.drop(gold.columns[[0,1,2,4]], axis=1)
gold.head()

,GLD
,close
time,
2017-05-25 00:00:00-04:00,119.50
2017-05-26 00:00:00-04:00,120.55
2017-05-30 00:00:00-04:00,120.15
2017-05-31 00:00:00-04:00,120.62
2017-06-01 00:00:00-04:00,120.66


In [8]:
#Concatenate Gold and Bitcoin Data and save to CSV
gold_btc = pd.concat([gold, btc], axis="columns", join="inner")
gold_btc.to_csv('Resources/gold_btc.csv')
gold_btc

,GLD,BTC
,close,close
